In [1]:
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
def fundamentals_from_yahoo(ticker):
    url="https://finance.yahoo.com/quote/%s/financials" % ticker
    soup = BeautifulSoup(urlopen(url))
    rows = soup.find_all(text='Gross Profit')[0].parent.parent.parent.parent.contents
    
    fundis=[]
    for row in rows:
        if len(row.contents) > 3:
            try:
                key = row.contents[0].contents[0].text
                if key == 'Revenue':
                    fundis.append(["Date"] + 
                                  [datetime.strptime(col.contents[0].text, '%m/%d/%Y').date()
                                   for col in row.contents[1:]])
                else:
                    fundis.append([key] + [
                        int(col.contents[0].text.replace(',',''))/1e6 
                        for col in row.contents[1:]])
            except:
                pass

    fundis = np.transpose(fundis)
    df = pd.DataFrame(data=fundis[1:, 1:], columns=fundis[0, 1:], index=fundis[1:, 0])
    #df.index.name = "Date"
    return df

In [3]:
fundis = fundamentals_from_yahoo('AAPL')
fundis

IndexError: too many indices for array